Aufgabe 1: [15/120 Punkte] Nenne 5 **verschiedene** Anwendungen wo Key-Value-Datenbanken Vorteile gegenüber Relationalen Datenbanken haben, und beschreibe spezifisch (aber dennoch stichwortartig) für jeden Fall, was genau der jeweilige Vorteil ist (und: was potentielle Nachteile sind)

In [ ]:
df=spark.read.csv("supermarket_sales.csv", header=True)
rename_df2 = df.withColumn("Gender", when(df.Gender == "male","m")
                                 .when(df.Gender == "female","f")
                                 .when(df.Gender == "Female","f")
                                 .when(df.Gender == "Male","m")  
                                 .otherwise(df.Gender))
rename_df2.show()

df3_count=rename_df2.groupBy("Gender").count()
df3_count.show()

df=spark.read.csv("supermarket_sales.csv", header=True)

from pyspark.sql.functions import when, col

rename_df2 = df.withColumn("Branch", when(df.Branch == "A","Yangon")
                                 .when(df.Branch == "B","Mandalay")
                                 .when(df.Branch == "C","Naypyitaw") 
                              .otherwise(df.Branch))
rename_df2.show()

sum_gross_income = rename_df2.groupBy("Branch").agg(_sum("Total"))

sum_gross_income.show()

rename_df3 = df.withColumn("Payment", when(df.Payment == "creditcard","Creditcard")
                                 .when(df.Payment == "credit card","Creditcard")
                                 .when(df.Payment == "Credit card","Creditcard") 
                                 .when(df.Payment == "cash","Cash")
                                 .when(df.Payment == "ewallet","Ewallet") 
                              .otherwise(df.Payment))
rename_df3.show()

count_function = rename_df3.groupBy("Payment").count()

count_function.show()

Aufgabe 2: [15/120 Punkte] erkläre was denn nun wirklich die Hauptunterschiede zwischen einem Data Warehouse und einem Data Lake sind und gib Beispiele für Anwendungen in denen entweder ein Data Warehouse oder ein Data Lake die bessere Wahl ist

Aufgabe 3: [15/120 Punkte] im folgenden Code hat sich der Fehlerteufel eingeschlichen. Finde die 5 Fehler (10 Punkte), und beschreibe stichwortartig, was der Code denn tun würde, wenn keine Fehler drin wären. (5 Punkte)

In [ ]:
from bson.son import SON
client = MongoClient("localhost", 27017)
connect = client["database"],["collection"]

pipe2 = [ { '$mismatch' : { 'origin' : 'ATL',
                         'dest' : 'BOS',
                         'dayofweek' : 3
                       }
          },
          { '$group' : { '_id' : { 'origin' : '$origin',
                                   'destination' : '$dest'
                                 },
                         'Failure' : { '$sum' : { '$cond' : [{ '$eq' : ['$cancelled', 1]}, 1, 0 ]} },
                         'Success' : { '$sum' : { '$cond' : [{ '$eq' : ['$cancelled', 0]}, 1, 0 ]} },
                         'Total' : { '$sum' : 1 }
                        }
           },
           { '$project' : { 'Failure' : 1,
                            'Success' : 1,
                            'Total' : 1,
                            'FailPercent' : { '$divide' : [ '$Failure', '$Total' ] }
                          }
           },
           # im SON-Statement ist kein Fehler, das gehoert so 
           { '$sort' : SON([('_id.origin', 1), ('_id.destination', 1 )]) }
         

result = connection.aggregate(pipeline=pipe2)
print(result)

Aufgabe 4: [15/120 Punkte] Erkläre was lazy evaluation ist an Hand von 2 in den Vorlesungen behandelten Technologien/Ansätzen (1 Lazy, eine nicht lazy) - und erläutere, was die Auswirkungen in der Praxis sind.

Aufgabe 5: [40/120 Punkte]
    In Moodle findest Du Super-Markt Daten. Werte diese in Spark aus, und finde heraus:
    
    1) wie viele Einkäufe wurden von Frauen, bzw von Männern getätigt?
    2) wie viel Umsatz wurde an den 3 Standorten jeweils getätigt - gib diesen zusammen mit dem NAMEN des jeweiligen Standorts aus
    3) welche Bezahlart die höchste durchschnittliche User-Satisfaction hat
    
Exportiere die Ergebnisse von Teilaufgabe 3 als CSV-Datei - stelle dabei sicher, dass unabhängig von der Größe des Datensatzes nur 1 einzige CSV-Datei generiert wird.

Beschreibe für jede der Teilaufgaben wie Du die Daten dazu aufbereiten würdest (Stichwort: Designentscheidungen)  

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Supermarkt_sales") \
      .getOrCreate() 
spark

df=spark.read.format("csv").option("header","true").load("supermarket_sales.csv")
df.show()

df2=df.groupBy("Gender").count()
df2.show()

df3 = df.withColumn("Gender", when(df.Gender == "Female", "f")
                              .when(df.Gender == "Male","m")
                              .when(df.Gender == "female", "f")
                              .when(df.Gender == "male", "m")
                              .otherwise(df.Gender))
df4 = df3.groupBy("Gender").count()
df4.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|     f|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|     null|       Normal|  Male|  Ho

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
#from pyspark.sql.functions import when, col
#from pyspark.sql.functions import sum as _sum

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Supermarkt_sales") \
      .getOrCreate() 
spark

df_new = spark.read.format("csv").option("header","true").load("supermarket_sales.csv")
df.show()

modified_branch_df = df_new.withColumn("Branch", when(df_new.Branch == "A", "Yangon")
                                                .when(df_new.Branch == "B", "Mandalay")
                                                .when(df_new.Branch == "C", "Naypyitaw")
                                                .otherwise(df_new.Branch))
income_per_city = modified_branch_df.groupBy("Branch").agg(sum("Total").alias("sum_salary"))

income_per_city.show()


+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|     f|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|     null|       Normal|  Male|  Ho

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Supermarkt_sales") \
      .getOrCreate() 
spark

df_new_2=spark.read.csv("supermarket_sales.csv", header=True)




df_modified_table = df_new_2.withColumn("Payment", when(df_new_2.Payment == "creditcard", "Creditcard")
                                                .when(df_new_2.Payment == "credit card", "Creditcard")
                                                .when(df_new_2.Payment == "Credit Card", "Creditcard")
                                                .when(df_new_2.Payment == "cash", "Cash")
                                                .when(df_new_2.Payment == "ewallet", "Ewallet")
                                                .otherwise(df_new_2.Payment))
df_modified_table.show()

bla = df_modified_table.groupBy("Payment").count()
bla.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date| Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|13:08|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|     f|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|10:29|       Cash|  76.4|            4.761904762|        3.82|   9.6|
|631-41-3108|     A|     null|       Normal|  Male|  Ho

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Supermarkt_sales") \
      .getOrCreate() 
spark

df_product = spark.read.csv("supermarket_sales.csv", header=True)

df_rated = df_product.groupBy("Product Line").count()
df_rated.show()


+--------------------+-----+
|        Product Line|count|
+--------------------+-----+
|  Home and lifestyle|   43|
| Fashion accessories|   41|
|   Health and beauty|   44|
|Electronic access...|   40|
|  Food and beverages|   43|
|   Sports and travel|   44|
+--------------------+-----+



Aufgabe 6: [20/60 Punkte] Ein regionales Unternehmen hat ein großes Investment durch eine Venture Capital Firma bekommen, und möchte jetzt international expandieren. 

Euer Auftrag ist es das Unternehmen zum Thema "Datenbanken" (die für die internationale Expansion benötig werden) zu beraten.

Das bisherige Hauptprodukt des Unternehmens sind mit Schokolade überzogene Fruchtgummis. Im Zuge der Expansion möchte das Unternehmen aber auch gleichzeitig neue Produkte einführen, bzw an regionale Geschmäcker anpassen. Dadurch, dass die Expansion überall gleichzeitig erfolgen soll, sind am Anfang wenig Möglichkeiten direkt Kundenumfragen zu machen - statt dessen soll die Resonanz auf Marketingcampagnen und die Interaktion mit der jeweiligen Landeswebseite genutzt werden, um lokale "Geschmäcker" und damit das Potential für weitere Produkte zu erforschen.

Der bisher einzige Vertriebskanal ist eine Online-Präsenz, zumindest anfänglich soll dies auch so bleiben, jedoch mit potentiell separaten Webseiten für verschiedene Länder/Regionen. Hierzu besteht aber noch keine fest definierte Strategie, der Unternehmer würde von Euch auch gerne wissen, was für Möglichkeiten, Vor- und Nachteile, und Risiken verschiedener Ansätze (aus Sicht der Nutzung von Datenbanken) sind.

Erläutere **stichwortartig**:

1) welche Technologien in welchem Bereich eingesetzt werden, und warum
2) was für Strategien, bzw Tools Ihr einsetzen wollt um 
    - die schon vorhandenen Daten zu migrieren
    - neue Daten mit den alten zu verbinden
    - benötigte Daten zu erhalten
3) welche Strategien sich anbieten für das Reporting, die Datenhaltung, Backups etc.
4) was die Auswirkungen verschiedener Expansionsstrategien (z.B. Marketing, Website etc) auf die Nutzung/Nützlichkeit/Performance der Datenbank(en) wäre 
5) Ob es "Bottlenecks" oder mit (hohem) Risiko verbundene Datenbank-Komponenten gibt, und wie man damit in der Praxis umgehen soll

(wirklich nur stichwortartig, keine Romane!)